In [1]:
%%time
import json
from unstructured.partition.auto import partition
import unstructured
import pickle
import os
# from langchain_text_splitters import RecursiveCharacterTextSplitter

CPU times: total: 703 ms
Wall time: 4.02 s


In [2]:
def load_pdf_name(hash_id):
    resources = json.load(open('./resources/res.json'))
    pdfname = list(filter(lambda x:x['hash'] == hash_id,resources))[0]['filename']
    return "./resources/"+pdfname
def load_pdf(hash, cache=True):
    pklfn = './resources/'+hash+".pkl"
    if os.path.isfile(pklfn) and cache:
        pages = pickle.load(open(pklfn, 'rb'))
        return pages
    fname = load_pdf_name(hash)
    elements = partition(fname)
    pages = list(range(elements[-1].metadata.page_number))

    for el in elements:
        if type(pages[el.metadata.page_number-1]) == int:
            pages[el.metadata.page_number-1] = []
        if type(el)==unstructured.documents.elements.Footer:
            continue
        if type(el)==unstructured.documents.elements.Header:
            continue
        pages[el.metadata.page_number-1].append(el)
    pickle.dump(pages,open(pklfn, 'wb'))
    return pages
def prep_documents_set(pdf_pages):
    docs = []
    for p in pdf_pages:
        str = ""
        # print(type(p))
        for el in p:
            str += el.text+"\n"
        docs.append(str)
    return docs
def docs_overlap(_docs):
    docs = _docs.copy()
    maxl = len(docs)
    splitter_char = ". "
    overlap_num_sentences = 3
    for i, doc in enumerate(docs):
        if i<1:
            continue
        prechunk = _docs[i-1].split(splitter_char)[-overlap_num_sentences:]
        # if i == 9: print(prechunk)
        docs[i] = splitter_char.join(prechunk) + docs[i]
        if i>=maxl-1:
            continue
        pstchunk = _docs[i+1].split(splitter_char)[:overlap_num_sentences]
        docs[i] = docs[i] + splitter_char.join(pstchunk)
    return docs
    

In [9]:
%time
pages = load_pdf("0e3ad6ccde4c4675ac0fea7c6ee35bc7")

CPU times: total: 0 ns
Wall time: 0 ns


In [19]:
# pickle.dump(pages,open('./resources/857irwueifdkjwo9eruj2iroeakdfm.pkl', 'wb'))

In [62]:
"abcd efgh ijkl mnop qrst".split(' ')[-3:]

['ijkl', 'mnop', 'qrst']

In [10]:
docs = prep_documents_set(pages)
ov = docs_overlap(docs)
print(ov[0])


Project Title: RAG Document Q&A Chatbot Development
1. Introduction:
The purpose of this document is to outline the requirements for developing a RAG (Retrieval-Augmented Generation) Q&A chatbot with the capability to load and process data from various file formats, store embeddings in a local vector database, and provide a frontend UI for user interaction.
2. Project Overview:
The project involves the development of a RAG Q&A chatbot leveraging state-of-the-art techniques in natural language processing (NLP), machine learning and Generative AI. The Q&A chatbot will be capable of understanding user queries, retrieving relevant information from pre-loaded data files, and presenting the information in a user-friendly manner through a Streamlit-based frontend UI.
3. Functional Requirements:
Data Loading (one time):
The chatbot should be able to load data from various file formats including PDF, WORD, JSON, CSV, Key- value pair, EXCEL, etc. Upon loading, the data should be processed to ext

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import dotenv
dotenv.load_dotenv()

model = ChatOpenAI()

In [75]:


from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from typing import List

model = ChatOpenAI()



class PageSummary(BaseModel):
    summary:List[str] = Field(description="Summary of given page content as a list of string")
    precontext: str = Field(description="General Summary of the the page to use as context to provide summary for future pdfs. Generate this field such that the old precontext is preserved, but still, the context of this page dominates, so that the context is clearer for next page")
    pass
parser = PydanticOutputParser(pydantic_object=PageSummary)


precontext_template = PromptTemplate(
    template="""You're a content summarizer bot. From a book/pdf, Summarize the following one page content in bullet points: 
    
    {content}

    The context of the pdf till now is following, just to have an idea of what above content might mean:
    {precontext}

    Output Format Instruction:
    {format_instructions}
    
    """,
    input_variables=["content", "precontext"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


template = PromptTemplate(
    template="""You're a content summarizer bot. From a book/pdf, Summarize the following one page content in bullet points: 
    
    {content}

    Output Format Instruction:
    {format_instructions}
    
    """,
    input_variables=["content"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)



default_summarize_chain = template | model | parser

precontext_summarize_chain = precontext_template | model | parser


def summarize(txt, gen_summary=""):
    if gen_summary == "":
        return default_summarize_chain.invoke({"content":txt})
    return precontext_summarize_chain.invoke({"content":txt, "precontext":gen_summary})

inpu = """The attacker used duct tape to affix Čarek's hand, holding the gun, to his head. Now all he had to do was pull the trigger, and after confirming Čarek's death, remove the tape and any signs of a third party from the room. We don't know who the attacker
was, but he was certainly a professional.
The Salzburg police finally set their sights on the man who appeared at St. Ursula
Clinic at 2:00 AM the day after the murder. It had now been two weeks since Čarek's murder — but the only three people who could possibly identify this man had been killed with an axe...
The police believe there is a connection between the Čarek murder and the man with
the bullet wound who appeared at the hospital, but they do not recognize a link between this man and Kottmann. The fact that all three witnesses were murdered was an unfortunate coincidence. Their line of reasoning is this: the man who killed Čarek was a trained terrorist, and those types of people never have ties to serial killers. Kottmann was the type who acted on his desires and impulses, and not the kind of person who would commit murder at the request of another and commit suicide afterward, to seal the case — which makes sense.
But they have no answer for Kottmann's last words: "One, two, three... My mission is
complete."
An illustration from Dorn in the Darkness. With its wavering message that shifts between good and evil, the Dorn series has a fanatical teen readership, but due to its extreme violence and sexual material, is derisively called "the most enticing kind of pulp literature" by critics.
My own personal theory goes like this. The man who checked in to the hospital with the injured arm was the man who killed Čarek. He hoped that the hospital staff would not report his presence to the police, but he had a plan in case they did"""

outp = summarize(inpu)

print(outp)

summary=["The attacker used duct tape to affix Čarek's hand, holding the gun, to his head and then killed him", 'The police suspect a connection between the murder and a man with a bullet wound who appeared at the hospital', 'All three witnesses who could identify the man with the bullet wound were killed with an axe', 'The police believe the killer was a trained terrorist, not linked to serial killers like Kottmann', "Kottmann's last words hint at completing a mission, raising questions about his involvement", 'The Dorn series is popular among teens but criticized for its extreme violence and sexual content', 'Personal theory suggests the man with the injured arm who checked into the hospital was the killer'] precontext="The page discusses the aftermath of Čarek's murder, with the police investigating potential leads and connections between different individuals. It also delves into the controversial nature of the Dorn series and presents a personal theory regarding the identity of th